# Test incident reports
#### [TIR1](#TIR1)
* When VMOTOR turns on, there's parasitic power that feeds back to the gate of Q305, turning VMOTOR off.
* The main issue is that USB data lines are feeding into VUSB through D401.  Insert a schottky diode between VUSB and D401 to prevent this.
* Remove R117 because it makes a path from +3V3 to R115, Q100 into VDBG.  Removing R117 still allows the MCU to be programmed.

#### [TIR2](#TIR2)
* A parasitic current flows from VBAT to body diode of Q300, to R304, R303 through D106, D107, D108 , R110, R111, R112 to 3V3.  The leakage of D106, D107 and D108 causes the leakage.  To make matters worse, these diodes are in parallel, so their leakage currents add up.
* Because the BAV170 was not available, BAT54C diodes have been used.  BAT54C has a much higher leakage current.
* D106, D107 and D108 can be replaced by BC856B (emitter to pin 1 of BAT54, collector to pin 2 of BAT54, base to pin 3 of BAT54).  The power supply current in sleep mode is 0.6µA.

#### [TIR3](#TIR3)
* When the IO6 pin of the MCU is HiZ, it's not low.  This causes the motor to turn during programming.
* This voltage level on IO6 is odd, because the motor controller has internal pull down resistors.
* The voltage on the input of the motor controller must be <0.5V to keep it off.
* Adding a 10k pull down resistor, pulls the voltage on IO6 to 0.66V, which is turns the motor off in our case, but it's not a guarantee.
* A 3k3 pull down resistor (a value already present in the design) pulls the voltage on IO6 to 0.235V, which is low enough to keep the motor off.

# Power test

## Power enable by press of a momentary switch
* **Test:** Power on by switch
* **Expected result:**T Power on when switch pushed
* **Actual result:**T power turns on and off when switch pushed
* **Status:** ❌ [TIR1](#TIR1)

## Powered by battery
### Reverse polarity
* **Test:** Connect 6V lab power supply with reverse polarity
* **Expected result:** No damage, Vmotor remains at 0V
* **Actual result:** 
  * Vmotor remains at 1.2mV
  * When applying correct polarity, Vmotor is 5.99V and the MCU works again.
* **Status:** ✅

### Voltage drop over Q300 and Q301
* **Test:** 
  * Connect a 6V lab power supply to the battery terminals.
  * Disable 5V by pulling U301.4 low.
  * Apply a 500mA constant current load to VMOTOR.
  * Measure the voltage at the battery terminals and at the VMOTOR net.
* **Expected result:** The voltage at the battery terminals xxx mV higher than at the VMOTOR net.
  * rdsON is 45mohm for Vgs = 4.5V -> Voltage drop over Q300 and Q301 is 22.5mV at 500mA. -> 45mV total.
* **Actual result:** 
  * VBAT = 5.95V
  * VMOTOR = 5.89V
  * VBAT - VMOTOR = 60mV
  * Vgs(Q300) = 5.89 - 0.69 = 5.2V
* **Status:** 45mV is close enough to 60mV ✅

## Powered by debug port
* Power the board with the SKEDD connector only.

### VMOTOR voltage
* **Test:** Measure voltage on VMOTOR
* **Expected result:** VMOTOR is a diode drop lower than VDBG.
* **Actual result :** 
  * VDBG = 4.55V
  * VMOTOR = 4.05V
* **Status:** ✅
  * D300 has a 500mV voltage drop, which is significant when compared to a common schottky diode.  We trade off forward voltage with reverse leakage.

### Battery short circuit
* **Test:** short circuit the battery terminals
* **Expected result:** VMOTOR voltage should not change (i.e. remain a diode drop lower than VDBG)
* **Actual result:** 4.02V
* **Status:** ✅

## Powered by USB port
todo

## VRTC
* **Test:** Measure VRTC voltage when only battery is connected and MCU is not powered.
* **Expected result:** VRTC is between 1.8Vmin, 2.2Vtyp and 2.9Vmax
* **Actual result:** 2.11V
* **Status:** ✅

## Sleep current
* **Test:** Measure sleep current when only battery is connected and MCU is not powered and RTC is not outputting a square wave.
* **Expected result:** Sleep current around 500nA
* **Actual result:** 1.3µA
* **Status:** ❌ [TIR2](#TIR2)

## Communication test
### USB VID:PID
* **Test:** Get USB VID:PID
* **Expected result:** ❓
* **Actual result:** 303A:0001 Espressif USB JTAG/serial debug unit
* **Status:** ✅

### USB serial
* **Test:**
  * Connect to USB serial port and send and receive data
  * Enable USB-CDC in platformio.ini with '-DARDUINO_USB_MODE=1'
* **Expected result:** Receive data and send data works
* **Actual result:** Received data and sent data works
* **Status:** ✅

## Peripheral test
### Voltage monitor for VMOTOR
* **Test:** measure VMOTOR with ADC of the MCU
* **Expected result :** within 1% of VMOTOR
* **Actual result:**
  * ADC = 4048mV 
  * DMM = 4.02V
* **Status:** ✅

### Motor
#### Motor VREF
* **Test:** Measure VREF of the motor driver
* **Expected result:** 2.25V
* **Actual result:** 2.24V
* **Status:** ✅

#### Motor inputs
* **Test:** Connect motor inputs to MCU.  When MCU is powered and being programmed, the motor inputs should be high impedance.  Motor should not turn.
* **Expected result:** Motor does not turn
* **Actual result:** Motor turns
* **Status:** ❌ [TIR3](#TIR3)

### RTC
#### RTC frequency
* **Test:** Measure RTC frequency
* **Expected result:** 32.768kHz +/- 20ppm (based on 20ppm crystal)
* **Actual result:**

### Momentary switches
#### SW100
* **Test:** Press SW100 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 0.6V
* **Actual result:** 0.63V
* **Status:** ✅

#### SW101
* **Test:** Press SW101 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is  2.7V/2  + 0.6 = 1.95V
* **Actual result:** 1.96V
* **Status:** ✅

#### SW102
* **Test:** Press SW102 and measure SNS_BUTTON
* **Expected result:** SNS_BUTTON is 6.6/9.9 * 2.7V  + 0.6 = 2.4V
* **Actual result:** 2.4V
* **Status:** ✅

